In [1]:
# Ref: https://qiita.com/noca/items/00646efd9d4a7302f522
from ipywidgets import Textarea
import io
if 'open' in globals():
    del open
if 'input' in globals():
    del input
original_open = open
class custom_open():
    def __init__(self):
        self.text = ''
    def __call__(self, file, *args, **kwargs):
        if file == 0:
            return io.StringIO(self.text)
        return original_open(file, *args, **kwargs)
    def updater(self, change):
        self.text = change["new"]
class custom_input():
    def __init__(self):
        self.__sio = io.StringIO('')
        self.shell = get_ipython()
        if self.shell.events.callbacks['pre_run_cell'] != []:
            self.shell.events.callbacks['pre_run_cell'] = []
        self.shell.events.register('pre_run_cell', self.pre_run_cell)
    def __call__(self):
        return self.__sio.readline().strip()
    def pre_run_cell(self, info):
        text = self.shell.user_ns.get('text_area', None).value
        self.__sio = io.StringIO(text)
open = custom_open()
input = custom_input()
text_area = Textarea()
text_area.observe(open.updater, names='value')
display(text_area)

Textarea(value='')

In [46]:
import sys
sys.setrecursionlimit(10 ** 6)

N, Q = map(int, input().split())
A = list(map(int, input().split()))

siz = 1
while siz < N:
    siz *= 2

#b 1-index
node = [False] * 2 * siz

def update(pos, x):
    pos += siz
    node[pos] ^= x
    
    while pos >= 2:
        pos //= 2
        node[pos] = node[pos*2] ^ node[pos*2+1]

# u：セル番号
# [a, b)：セルに対応する区間
# [l, r)：求めたい区間の結果
def query(l, r, a=0, b=siz, u=1):
    if r <= a or b <= l:
        return False
    
    if l <= a and b <= r:
        return node[u]
    
    mid = (a + b) // 2
    vl = query(l, r, a, mid, u * 2)
    vr = query(l, r, mid, b, u * 2 + 1)
    return vl ^ vr

for i in range(N):
    update(i, A[i])

for _ in range(Q):
    t, x, y = map(int, input().split())
    if t == 1:
        x -= 1
        update(x, y)
    
    if t == 2:
        x -= 1
        ans = query(x, y)
        print(ans)

1
0
5
3
0


In [47]:
node

[False,
 2,
 0,
 2,
 5,
 5,
 2,
 False,
 2,
 7,
 5,
 0,
 2,
 False,
 False,
 False,
 7,
 5,
 3,
 4,
 7,
 2,
 0,
 0,
 5,
 7,
 False,
 False,
 False,
 False,
 False,
 False]

In [48]:
siz

16